<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session7_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 7 (Word Sequences) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [1]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/
pip install svgling
pip install python-crfsuite

Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 303 (delta 76), reused 42 (delta 27), pack-reused 190
Receiving objects: 100% (303/303), 293.96 KiB | 1.92 MiB/s, done.
Resolving deltas: 100% (166/166), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 6.6 MB/s eta 0:00:00


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scripts.compute_metrics import ComputeMetrics
from scripts.text_preprocessing import TextPreprocessing
from complementary_material.textserver import TextServer
from scripts.utils import pretty_table
from scipy.stats import pearsonr
import pandas as pd
pd.set_option("display.precision", 4)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
dt = pd.read_csv('complementary_material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('complementary_material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

## TODO
1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project. Compute their similarities by considering *lemmas* and *Jaccard distance*. <br>

2. Compare the results with those in session 2 (document structure) in which words were considered. : <br>

3. Compare the results with gold standard by giving the pearson correlation between them. <br>

4. Questions (justify the answers): <br>

      Which is better: words or lemmas?

      Do you think that could perform better for any pair of texts?

In [ ]:
import nltk
import svgling
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
sentence = "Mark Pedersen is working at Google since 1994."
res = nltk.ne_chunk(
        nltk.pos_tag(
          nltk.word_tokenize(sentence)))
type(res), res

In [ ]:
res

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
sentence = "Mark Pedersen works at Google since 1994."
doc = nlp(sentence)

[(token.text, token.pos_, token.tag_, token.lemma_, token.is_stop,
  token.ent_iob_, token.ent_type_) for token in doc]

In [ ]:
[(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
spacy.displacy.render(doc, jupyter=True, style='ent')

In [ ]:
with doc.retokenize() as retokenizer:
    tokens = [token for token in doc]
    for ent in doc.ents:
        retokenizer.merge(doc[ent.start:ent.end],
            attrs={"LEMMA": " ".join([tokens[i].text
                                for i in range(ent.start, ent.end)])})

[(token.text, token.pos_, token.tag_, token.lemma_, token.is_stop,
  token.ent_iob_, token.ent_type_) for token in doc]

In [ ]:
ts = TextServer('MarioROT', 'IHLTMAI23*', 'entities')
# ts.entities("Mark Pedersen works at Google since 1994.")

In [ ]:
ts = TextServer('MarioROT', 'IHLTMAI23*', 'tokenizer')

In [ ]:
ts.tokenizer('Men want children. They get relaxed with kids.')

In [ ]:
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),("dog", "NN"),\
            ("barked", "VBD"), ("at", "IN"), ("the", "DT"), ("cat", "NN")]
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
cp.parse(sentence)

In [ ]:
nltk.download('conll2000')
from nltk.corpus import conll2000

test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
sentence = conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99]
sentence

In [ ]:
from nltk import tree2conlltags
tree2conlltags(sentence)

In [ ]:
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.accuracy(test_sents))

In [ ]:
nltk.download('treebank')
train = nltk.corpus.treebank.tagged_sents()[:3000]
test = nltk.corpus.treebank.tagged_sents()[3000:]
model = nltk.tag.CRFTagger()
model.train(train,'/content/crfTagger.mdl')
model.accuracy(test)

In [ ]:
tagger = nltk.tag.CRFTagger()
tagger.set_model_file('crfTagger.mdl')
tagger.tag(['the', 'men', 'attended', 'to', 'the', 'meetings'])

In [ ]:
train[0]

In [ ]:
tagger = nltk.tag.CRFTagger()
tagger.set_model_file('/content/drive/My Drive/models/crfTagger.mdl')
tagger.tag(['the', 'men', 'attended', 'to', 'the', 'meetings'])

In [6]:
tp = TextPreprocessing()

# ----- Tokenization -----
# NLTK
dt[2] = tp.tokenize_data(list(dt[0]),'nltk')
dt[3] = tp.tokenize_data(list(dt[1]),'nltk')
# spaCy
dt[4] = tp.tokenize_data(list(dt[0]),'spacy')
dt[5] = tp.tokenize_data(list(dt[1]),'spacy')
# Data cleaning
dt[6]=tp.clean_data(list(dt[0]))
dt[7]=tp.clean_data(list(dt[1]))


# ----- Lemmatization -----
# -- With Tokens
# NLTK
dt[8]=tp.lemmatize_data(list(dt[0]),'nltk',False)
dt[9]=tp.lemmatize_data(list(dt[1]),'nltk',False)
# spaCy
dt[10]=tp.lemmatize_data(list(dt[0]),'spacy')
dt[11]=tp.lemmatize_data(list(dt[1]),'spacy')

# -- With Cleaned data
# NLTK
dt[12]=tp.lemmatize_data(list(dt[6]),'nltk')
dt[13]=tp.lemmatize_data(list(dt[7]),'nltk')
# spaCy
dt[14]=tp.lemmatize_data(list(dt[6]),'spacy')
dt[15]=tp.lemmatize_data(list(dt[7]),'spacy')


# ----- Word Desambiguation -----
# --- With Tokens
# NLTK
dt[16]= tp.wsd_lesk_data(list(dt[2]),'nltk', keep_failures=True, synset_word=True)
dt[17]= tp.wsd_lesk_data(list(dt[3]),'nltk', keep_failures=True, synset_word=True)
# Spacy
dt[18]= tp.wsd_lesk_data(list(dt[4]),'nltk', keep_failures=True, synset_word=True)
dt[19]= tp.wsd_lesk_data(list(dt[5]),'nltk', keep_failures=True, synset_word=True)
# Cleaned data
dt[20]= tp.wsd_lesk_data(list(dt[6]),'nltk', keep_failures=True, synset_word=True)
dt[21]= tp.wsd_lesk_data(list(dt[7]),'nltk', keep_failures=True, synset_word=True)

# --- With Lemmas
# NLTK Lemmas
dt[22]= tp.wsd_lesk_data(list(dt[8]),'nltk', keep_failures=True, synset_word=True)
dt[23]= tp.wsd_lesk_data(list(dt[9]),'nltk', keep_failures=True, synset_word=True)
# Spacy Lemmas
dt[24]= tp.wsd_lesk_data(list(dt[10]),'nltk', keep_failures=True, synset_word=True)
dt[25]= tp.wsd_lesk_data(list(dt[11]),'nltk', keep_failures=True, synset_word=True)
# Cleaned NLTK Lemmas
dt[26]= tp.wsd_lesk_data(list(dt[12]),'nltk', keep_failures=True, synset_word=True)
dt[27]= tp.wsd_lesk_data(list(dt[13]),'nltk', keep_failures=True, synset_word=True)
# Cleaned SpaCy Lemmas
dt[28]= tp.wsd_lesk_data(list(dt[14]),'nltk', keep_failures=True, synset_word=True)
dt[29]= tp.wsd_lesk_data(list(dt[15]),'nltk', keep_failures=True, synset_word=True)

The code for the tokenization is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/text_preprocessing.py

The code for jaccard distance computation is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/compute_metrics.py

In [15]:
# -- Jaccard computation
jaccard_pairs = {'jaccard_nltk_token':[2,3], 'jaccard_spacy_token':[4,5], 'jaccard_clean_token':[6,7], # Tokens
                 'jaccard_nltk_lemma':[8,9], 'jaccard_spacy_lemma':[10,11], 'jaccard_clean_nltk_lemma':[12,13], 'jaccard_clean_spacy_lemma':[14,15], # Lemmas
                 'jaccard_nltk_token_wsd':[16,17], 'jaccard_spacy_token_wsd':[18,19], 'jaccard_clean_token_wsd':[20,21], # WSD
                 'jaccard_nltk_lemma_wsd':[22,23], 'jaccard_spacy_lemma_wsd':[24,25], 'jaccard_clean_nltk_lemma_wsd':[26,27], 'jaccard_clean_spacy_lemma_wsd':[28,29] #WSD
                 }

for name, values in jaccard_pairs.items():
  dt[name] = ComputeMetrics(dt[values].to_numpy(), ['jaccard'], 1).do()['jaccard']

Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...


In [16]:
dt[['gs',*[col for col in dt.columns.to_list() if 'jaccard' in str(col)]]]

,gs,jaccard_nltk_token,jaccard_spacy_token,jaccard_clean_token,jaccard_nltk_lemma,jaccard_spacy_lemma,jaccard_clean_nltk_lemma,jaccard_clean_spacy_lemma,jaccard_nltk_token_wsd,jaccard_spacy_token_wsd,jaccard_clean_token_wsd,jaccard_nltk_lemma_wsd,jaccard_spacy_lemma_wsd,jaccard_clean_nltk_lemma_wsd,jaccard_clean_spacy_lemma_wsd
0,4.500,0.6538,0.6538,0.6923,0.6538,0.6000,0.6667,0.6667,0.7037,0.7037,0.6923,0.7500,0.6538,0.6923,0.6923
1,5.000,0.2143,0.2143,0.2857,0.0769,0.0769,0.0000,0.0000,0.0769,0.0769,0.0000,0.0769,0.0769,0.0000,0.0000
2,4.250,0.6087,0.6087,0.7273,0.6087,0.5455,0.7273,0.7273,0.6087,0.6087,0.7273,0.6087,0.5455,0.7273,0.7273
3,4.500,0.4545,0.4000,0.2500,0.4545,0.4000,0.2500,0.2500,0.4545,0.4000,0.2500,0.4545,0.4000,0.2500,0.2500
4,5.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,5.000,0.4500,0.4500,0.6364,0.4500,0.4500,0.6364,0.6364,0.5909,0.5909,0.6364,0.5909,0.5909,0.6364,0.6364
455,4.750,0.6429,0.7143,0.2500,0.6429,0.6154,0.2500,0.2500,0.5385,0.6154,0.2500,0.5385,0.8000,0.2500,0.2500
456,5.000,0.6000,0.6000,0.6667,0.6000,0.5263,0.6667,0.6667,0.6000,0.6000,0.6667,0.5263,0.5263,0.6667,0.6667
457,4.000,0.6818,0.6818,0.8000,0.6818,0.6190,0.8000,0.8000,0.6818,0.6818,0.8000,0.6818,0.6190,0.8000,0.8000


In [17]:
pearson_group = lambda group: [round(pearsonr(dt['gs'], 1-dt[column])[0], 3) for column in dt.columns if group == str(column).split('_')[-1]]
names_group = lambda group: [' '.join(column.split('_')[1:-1]).upper() for column in dt.columns if group == str(column).split('_')[-1]]

results = {'Tokenization': {'Category':names_group('token'),'Values':pearson_group('token')},
           'Lemmatization': {'Category':names_group('lemma'), 'Values':pearson_group('lemma')},
           'Word Sense Disambiguation': {'Category':names_group('wsd'), 'Values':pearson_group('wsd')}}

for name, dic in results.items():
  pretty_table(dic, name)


--- Results for Tokenization
+----------+--------+
| Category | Values |
+----------+--------+
|   NLTK   |  0.45  |
|  SPACY   | 0.461  |
|  CLEAN   | 0.468  |
+----------+--------+

--- Results for Lemmatization
+-------------+--------+
|   Category  | Values |
+-------------+--------+
|     NLTK    | 0.449  |
|    SPACY    | 0.477  |
|  CLEAN NLTK | 0.481  |
| CLEAN SPACY | 0.491  |
+-------------+--------+

--- Results for Word Sense Disambiguation
+-------------------+--------+
|      Category     | Values |
+-------------------+--------+
|     NLTK TOKEN    |  0.42  |
|    SPACY TOKEN    | 0.429  |
|    CLEAN TOKEN    |  0.48  |
|     NLTK LEMMA    | 0.417  |
|    SPACY LEMMA    | 0.422  |
|  CLEAN NLTK LEMMA | 0.482  |
| CLEAN SPACY LEMMA | 0.493  |
+-------------------+--------+


# Conclusion

In this lab session excercise, we compared previous sentence pairs using Jaccard distance with and without lemmatization and data cleaning. And now with the Word Sense Desambiguation process, using both the "raw" lemmas and the lemmas with a previous data cleaning.

We can see in the table above that even if it is counter-intuitive the results of the pearson correlation between the previous lab (lemmas and words) outperformed the ones with the WSD both with words and lemmas, and with and without the data cleaning process.


